# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 3 2023 11:19AM,256614,19,ADV80011399,"AdvaGen Pharma, Ltd",Released
1,Feb 3 2023 11:19AM,256614,19,ADV80011400,"AdvaGen Pharma, Ltd",Released
2,Feb 3 2023 11:19AM,256614,19,ADV80011401,"AdvaGen Pharma, Ltd",Released
3,Feb 3 2023 11:19AM,256614,19,ADV80011402,"AdvaGen Pharma, Ltd",Released
4,Feb 3 2023 11:19AM,256614,19,ADV80011403,"AdvaGen Pharma, Ltd",Released
5,Feb 3 2023 11:19AM,256614,19,ADV80011404,"AdvaGen Pharma, Ltd",Released
6,Feb 3 2023 11:19AM,256614,19,ADV80011405,"AdvaGen Pharma, Ltd",Released
7,Feb 3 2023 11:19AM,256614,19,ADV80011406,"AdvaGen Pharma, Ltd",Released
8,Feb 3 2023 11:19AM,256614,19,ADV80011407,"AdvaGen Pharma, Ltd",Released
9,Feb 3 2023 11:19AM,256614,19,ADV80011408,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
34,256607,Released,1
35,256609,Released,16
36,256610,Released,17
37,256613,Released,1
38,256614,Released,21


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256607,NaN,NaN,1.0
256609,NaN,NaN,16.0
256610,NaN,NaN,17.0
256613,NaN,NaN,1.0
256614,NaN,NaN,21.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256513,0.0,1.0,0.0
256515,0.0,1.0,0.0
256531,0.0,0.0,1.0
256537,0.0,0.0,1.0
256551,0.0,0.0,3.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256513,0,1,0
256515,0,1,0
256531,0,0,1
256537,0,0,1
256551,0,0,3


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256513,0,1,0
1,256515,0,1,0
2,256531,0,0,1
3,256537,0,0,1
4,256551,0,0,3


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256513,,1,
1,256515,,1,
2,256531,,,1
3,256537,,,1
4,256551,,,3


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd"
21,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd"
22,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd"
39,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.
55,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc."
56,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation
60,Feb 3 2023 10:16AM,256605,10,ISDIN Corporation
115,Feb 3 2023 10:10AM,256604,15,"Person & Covey, Inc."
120,Feb 3 2023 10:10AM,256603,15,"Person & Covey, Inc."
136,Feb 3 2023 10:02AM,256601,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",,,21
1,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",,,1
2,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",,,17
3,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,,,16
4,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",,,1
5,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,,2,2
6,Feb 3 2023 10:16AM,256605,10,ISDIN Corporation,,,55
7,Feb 3 2023 10:10AM,256604,15,"Person & Covey, Inc.",,1,4
8,Feb 3 2023 10:10AM,256603,15,"Person & Covey, Inc.",,7,9
9,Feb 3 2023 10:02AM,256601,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",21,,
1,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1,,
2,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17,,
3,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16,,
4,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1,,
5,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,2,2,
6,Feb 3 2023 10:16AM,256605,10,ISDIN Corporation,55,,
7,Feb 3 2023 10:10AM,256604,15,"Person & Covey, Inc.",4,1,
8,Feb 3 2023 10:10AM,256603,15,"Person & Covey, Inc.",9,7,
9,Feb 3 2023 10:02AM,256601,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",21,,
1,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1,,
2,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17,,
3,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16,,
4,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",21.0,NaN,NaN
1,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
2,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17.0,NaN,NaN
3,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16.0,NaN,NaN
4,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",21.0,0.0,0.0
1,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
2,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17.0,0.0,0.0
3,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16.0,0.0,0.0
4,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2309294,106.0,68.0,0.0
12,513028,0.0,2.0,0.0
15,1539587,46.0,22.0,0.0
16,256586,1.0,0.0,0.0
19,1026420,42.0,11.0,1.0
21,1282868,5.0,0.0,0.0
22,513187,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2309294,106.0,68.0,0.0
1,12,513028,0.0,2.0,0.0
2,15,1539587,46.0,22.0,0.0
3,16,256586,1.0,0.0,0.0
4,19,1026420,42.0,11.0,1.0
5,21,1282868,5.0,0.0,0.0
6,22,513187,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,106.0,68.0,0.0
1,12,0.0,2.0,0.0
2,15,46.0,22.0,0.0
3,16,1.0,0.0,0.0
4,19,42.0,11.0,1.0
5,21,5.0,0.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,106.0
1,12,Released,0.0
2,15,Released,46.0
3,16,Released,1.0
4,19,Released,42.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Executing,68.0,2.0,22.0,0.0,11.0,0.0,0.0
Released,106.0,0.0,46.0,1.0,42.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,68.0,2.0,22.0,0.0,11.0,0.0,0.0
2,Released,106.0,0.0,46.0,1.0,42.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Completed,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Executing,68.0,2.0,22.0,0.0,11.0,0.0,0.0
2,Released,106.0,0.0,46.0,1.0,42.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()